# Pipeline setup. 
Open IAM role, and add permission Lambda full access to this sagemaker role. (you'll also need to grant sagemaker full access to lambda)

In [2]:
import sagemaker
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import TrainingStep, CreateModelStep
from sagemaker.lambda_helper import Lambda
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.lambda_step import LambdaStep, LambdaOutput, LambdaOutputTypeEnum
from sagemaker.jumpstart.estimator import JumpStartEstimator
from sagemaker.model import Model
from sagemaker.workflow.pipeline_context import PipelineSession


aws_region = "ap-northeast-1"
sess = PipelineSession()
role = sagemaker.get_execution_role()

model_id = "tensorflow-tc-bert-multi-cased-L-12-H-768-A-12-2"
instance_type = "ml.m5.4xlarge"
training_data = "s3://sagemaker-automated-execution-xxxxxxxxxx-ap-northeast-1/training-datasets/SST/"
endpoint_name = "endpoint_test1"


estimator = JumpStartEstimator(
    model_id=model_id,
    hyperparameters={
        "epochs": "1",
        "batch_size": "64",
        "use_fp16": "True",
        "train_only_top_layer": "True"
        },
    instance_type=instance_type,
    role=role,
    sagemaker_session=sess
    )

training_data_param = ParameterString(name="TrainingDataS3", default_value=training_data)

training_step = TrainingStep(
    name="TrainBERTModel",
    estimator=estimator,
    inputs={"training": training_data_param}
    )

# Create model with direct link to training artifacts
from sagemaker.jumpstart.model import JumpStartModel
model = JumpStartModel(  
    model_id=model_id,
    model_data=training_step.properties.ModelArtifacts.S3ModelArtifacts,
    role=role,
    sagemaker_session=sess
    )

# Create model step without specifying inputs
model_step = CreateModelStep(
    name="CreateModel",
    model=model
    )

# Lambda function ARN (Replace with actual Lambda ARN)
# Open IAM in console, access the role used by Lambda, add sagemaker full access policy.
lambda_function = Lambda(function_arn="arn:aws:lambda:ap-northeast-1:xxxxxxxxxxx:function:endpointTest1")

# Lambda step to deploy the model
lambda_step = LambdaStep(
    name="DeployEndpoint",
    lambda_func=lambda_function,
    inputs={
        "model_name": model_step.properties.ModelName,
        "endpoint_name": endpoint_name,
        "instance_type": instance_type,
        },
    outputs=[LambdaOutput(output_name="EndpointStatus", output_type=LambdaOutputTypeEnum.String)]
    )

pipeline = Pipeline(
    name="BERTDeploymentPipeline",
    parameters=[training_data_param],
    steps=[training_step, model_step, lambda_step]
    )

pipeline.upsert(role_arn=role)


Using model 'tensorflow-tc-bert-multi-cased-L-12-H-768-A-12-2' with wildcard version identifier '*'. You can pin to version '3.0.8' for more stable results. Note that models may have different input/output signatures after a major version upgrade.
No instance type selected for inference hosting endpoint. Defaulting to ml.p3.2xlarge.
INFO:sagemaker.jumpstart:No instance type selected for inference hosting endpoint. Defaulting to ml.p3.2xlarge.
/opt/conda/lib/python3.11/site-packages/sagemaker/workflow/lambda_step.py:165: UserWarning: Lambda function won't be updated because zipped_code_dir                 or script is not provided.
  warnings.warn(


{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-1:533267358966:pipeline/BERTDeploymentPipeline',
 'ResponseMetadata': {'RequestId': '11f0f996-3917-4223-b059-81924f602118',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '11f0f996-3917-4223-b059-81924f602118',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '95',
   'date': 'Sat, 08 Mar 2025 04:27:30 GMT'},
  'RetryAttempts': 0}}

In [14]:
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:ap-northeast-1:533267358966:pipeline/BERTDeploymentPipeline/execution/rbqhnbpy5nnp', sagemaker_session=<sagemaker.session.Session object at 0x7f6e7dc5eb50>)